In [6]:
%pip install langchain langchain-openai python-dotenv

  Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   ---------------------------------------- 894.7/894.7 kB 7.6 MB/s eta 0:00:00

   -- -------------------------------------  1/20 [urllib3]
   -- -------------------------------------  1/20 [urllib3]
   -- -------------------------------------  1/20 [urllib3]
   -- -------------------------------------  1/20 [urllib3]
   -- -------------------------------------  1/20 [urllib3]
   -- -------------------------------------  1/20 [urllib3]
   ---- -----------------------------------  2/20 [tenacity]
   ---- -----------------------------------  2/20 [tenacity]


In [2]:

from dotenv import load_dotenv
load_dotenv()  # .env 파일 내용을 환경변수로 로드
import os
os.getenv('OPENAI_API_KEY')[:4]


'sk-p'

```
LangChain의 핵심 컴포넌트 : 모델 호출단계를 구성하는 추상화 요소를 제공
    -PromptTemplate : LLM에 보낼낼 입력 프롬프트
    -ChatOpenAI : openai의 GPT- 모델 호출출 
    -Runnable : 실행가능한 객체에 대한 공통 인터페이스 -> invoke() 메서드를 통해 입력 -> 출력을 지원
    -StrOutPutParser : 문자열 출력 파서
파이프로 연경이 가능 .. ex) promt \ llm \ strparser

```

In [ ]:
from langchain_core.prompts import PromptTemplate
template = "{product} 제품을 생산하는 회사 이름을 뭘로 하면 좋을까?"
prompt = PromptTemplate.from_template(template)
formated_prompt = prompt.format(product = '커피')
formated_prompt

'커피 제품을 생산하는 회사 이름을 뭘로 하면 좋을까?'

LLM에서 invoke를 사용하는 이유는 모델에게 특정 입력을 주고 그 결과를 직접 요청하여 받기 위함입니다. 즉, "LLM을 한 번 호출해서 결과를 얻는" 행동입니다.

🔍 더 구체적으로 설명하면:
invoke는 보통 다음과 같은 상황에서 사용됩니다:

상황	설명
1. 직접 응답 받기	예: "다음 문장을 영어로 번역해줘" → 한 번 요청하고 결과를 받음
2. 프롬프트 테스트	여러 프롬프트 실험 중, 하나씩 직접 호출해서 반응을 확인할 때
3. 스트리밍 아님	invoke()는 전체 결과를 한 번에 받아오는 방식 (vs stream()은 결과가 점진적으로 도착함)
4. Chain 또는 Agent 구성 없이 단순 호출	LangChain 등의 구조를 쓰지 않고 단일 질문을 보낼 때
5. 비동기 작업이 아닌 동기 처리	즉시 결과가 필요한 경우

🧠 왜 굳이 invoke로 하나요?
프롬프트 하나에 대해 한 번만 응답을 받고 싶을 때
LLM을 체인(chain) 없이 직접 사용하는 경우
테스트나 실험할 때 빠르게 확인하려고
함수로 래핑하거나 반복 호출하려는 구조에서 명시적으로 "이때 호출"을 원할 때

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini', temperature = 0) #Runnable 객체 -> invoke()   머신러닝의 fit 같이  invoke로씀
response = llm.invoke([('human','안녕')])
response      #리스트 튜플로인수 넣어줌

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_92e0377081', 'id': 'chatcmpl-BbG7WFsb30pATpDozfQLRalnffeVF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f11fd46-c1be-4606-b446-b5406555eb72-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
# 문자열 출력 파서 Runnable 객체 - invoke
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_text = parser.invoke(response)
parsed_text 

'안녕하세요! 어떻게 도와드릴까요?'

여기까지 프로프트 만들고 llm만들어 llm실행하고  어디에 넣어서 테스트 뽑음음

LangChain Expression Language(LCEL)단일 체인 실행


In [ ]:
chain = prompt | llm | parser  # runnable 객체 -> invoke
result = 